In [1]:
import pandas as pd
import ast
from scipy.optimize import curve_fit
import numpy as np

COLOR = '#343434'

color_map = {
    "Zambia" : "#009E73", # green
    "Mozambique" : "#56B4E9", # blue
    "Nigeria" : "#0072B2",  # reddish-purple
    "Cameroon" : "#D55E00", # brown 
    "Democratic Republic of the Congo" : COLOR, #Black
    "Uganda" : "#F0E442",
    "Malawi" : "#E69F00", # orange 
    "Other" : "#C8C8C8" # grey
}

# Estimate total lineages
This notebook is used to perform rarefaction analysis on the number of introductions identified in each country.

First, we load the metadata for all sequences included in the analysis. We need this to get the collection dates of taxa in the tree.

In [2]:
md = list()
for file in ["supplemental_data1.csv", "supplemental_data2.csv"]:
    df = pd.read_csv( "../../data/" + file, usecols=["taxa", "country", "included_analysis", "collection_date"] )
    df["workshop"] = (file == "supplemental_data1.csv")
    md.append( df )
    
md = pd.concat( md )
md = md.loc[md["included_analysis"]].copy()
md.head()

Next, we expand the introduction table so thats we can randomly sample sequences linked to introductions (i.e. the introduction table is formatted as introductions with a list of descendents and we want a dataframe of descendents linked to introductions). Introductions are generated by the script in `analysis/estimate-introductions/estimate-introductions.py`.

In [3]:
intros = pd.read_csv( "../estimate-introductions/introductions.csv" )
	
longform = {
	"introduction" : [],
	"location" : [],
	"taxa" : [],
	"date" : [],
	"tree" : [],
}

for tree in intros["tree"].unique():
	tree_intros = intros.loc[intros["tree"]==tree]
	count = 0
	for _, row in tree_intros.iterrows():
		intro_name = f"{row['Location']}-{row['te']}-{count}"
		
		if row["children.names"].startswith("["):
			children_list = ast.literal_eval( row["children.names"] )
			children_dates = ast.literal_eval( row["children.dates"] )
		else:
			child_name = row["children.names"]
			children_list = [child_name]
			children_dates = [md.loc[md["taxa"]==child_name,"collection_date"].values[0]]
		
		for child, date in zip( children_list, children_dates ):
			longform["introduction"].append( intro_name )
			longform["taxa"].append( child )
			longform["date"].append( date )
			longform["location"].append( row["Location"] )
			longform["tree"].append( tree )
		count += 1
longform = pd.DataFrame( longform )
print( longform.shape )
longform.head()

(988397, 5)


,introduction,location,taxa,date,tree
0,Nigeria-T9-0,Nigeria,Africa|NGA|ERR998692|T9|1996-01-01,1996.0,100
1,Nigeria-T9-0,Nigeria,Africa|NGA|ERR998706|T9|1996-01-01,1996.0,100
2,Nigeria-T9-0,Nigeria,Africa|NGA|ERR1879656|T9|1997-01-01,1997.0,100
3,Nigeria-T9-0,Nigeria,Africa|NGA|ERR998707|T9|1996-01-01,1996.0,100
4,Nigeria-T9-0,Nigeria,Africa|NGA|ERR998705|T9|1996-01-01,1996.0,100


We perform 1000 trials of the rarefaction analysis for each country. Each trial consists of randomly sampling a tree, and then iteratively sampling 1-N sequences, where N is the maximum number of sequences collected from a country, and with each sampling counting the number of introductions identified. We then fit the rarefaction curve with a Michaelis-Menten kinetics model. The 1000 trials is used to calculate 95% confidence intervals on the parameters of the MM kinetics model.

In [4]:
def mm( x, V: float, M: float ):
	return ( V * x ) / ( M + x )

all_results = list()
all_params = list()
for country in ["Cameroon", "Democratic Republic of the Congo", "Malawi", "Mozambique", "Nigeria", "Uganda", "Zambia"]:
	subset = longform.loc[longform["location"] == country]
	params = {
		"country" : country,
		"V" : [],
		"Km" : [],
		"max_intros" : []
	}
	for trial in range( 1000 ):
		tree = np.random.choice( intros["tree"].unique() )
		trial_subset = subset.loc[subset["tree"]==tree]
		
		results = {
			"sequences" : [],
			"introductions" : [],
			"location" : country,
			"tree" : tree
		}	
		
		for i in range( 1, len( trial_subset ) - 1 ):
			boot = trial_subset.sample( n=i, replace=False )
			found = boot["introduction"].nunique()
			results["sequences"].append( i )
			results["introductions"].append( found )
		results = pd.DataFrame( results )
		all_results.append( results )
		fit, covar = curve_fit(
			f=mm,
			xdata=results["sequences"],
			ydata=results["introductions"],
			p0=[results["introductions"].max(),results["introductions"].max()],
			bounds=([results["introductions"].max(),1], [np.inf, np.inf])
		)
		
		params["V"].append( fit[0] )
		params["Km"].append( fit[1] )
		params["max_intros"].append( results["introductions"].max() )
	params = pd.DataFrame( params )
	all_params.append( params )

all_results = pd.concat( all_results )
all_results.to_csv( "rarefaction_curve.csv", index=False )
all_params = pd.concat( all_params )
all_params["percent_captured"] = all_params["max_intros"] / all_params["V"]
all_params.to_csv( "rarefaction-model-fit.csv", index=False )